Los autocodificadores son redes neuronales artificiales capaces de aprender representaciones densas de los datos de entrada, llamadas representaciones o codificaciones latentes, sin ninguna supervisión (es decir, el conjunto de entrenamiento no está etiquetado). Estas codificaciones suelen tener una dimensionalidad mucho más baja que los datos de entrada, lo que hace que los autocodificadores sean útiles para la reducción de la dimensionalidad (ver Capítulo 8), especialmente para fines de visualización. Los autocodificadores también actúan como detectores de características, y se pueden utilizar para el preentrenamiento no supervisado de redes neuronales profundas (como discutimos en el capítulo 11). Por último, algunos autocodificadores son modelos generativos: son capaces de generar aleatoriamente nuevos datos que se parecen mucho a los datos de entrenamiento. Por ejemplo, podrías entrenar a un autocodificador en imágenes de caras, y entonces podría generar nuevas caras.

Las redes adversarias generativas (GAN) también son redes neuronales capaces de generar datos. De hecho, pueden generar imágenes de caras tan convincentes que es difícil creer que las personas que representan no existan. Puedes juzgarlo por ti mismo visitando https://thispersondoesnotexist.com, un sitio web que muestra caras generadas por una arquitectura GAN llamada StyleGAN. También puedes consultar https://thisrentaldoesnotexist.com para ver algunos anuncios generados de Airbnb. Los GAN ahora se utilizan ampliamente para la súper resolución (aumentar la resolución de una imagen), la coloración, la potente edición de imágenes (por ejemplo, reemplazar los bombarderos fotográficos con un fondo realista), convertir bocetos simples en imágenes fotorrealistas, predecir los siguientes fotogramas en un vídeo, aumentar un conjunto de datos (para entrenar a otros modelos), generar otros tipos de datos (como texto, audio y series temporales), identificar las debilidades en otros modelos para fortalecerlos y más.

Una adición más reciente al grupo de aprendizaje generativo son los modelos de difusión. En 2021, lograron generar imágenes más diversas y de mayor calidad que las GAN, a la vez que fueron mucho más fáciles de entrenar. Sin embargo, los modelos de difusión son mucho más lentos de ejecutar.

Los autocodificadores, los GAN y los modelos de difusión no están supervisados, todos aprenden representaciones latentes, todos se pueden utilizar como modelos generativos y tienen muchas aplicaciones similares. Sin embargo, funcionan de manera muy diferente:

- Los autocodificadores simplemente aprenden a copiar sus entradas a sus salidas. Esto puede sonar como una tarea trivial, pero como verás, restringir la red de varias maneras puede hacerlo bastante difícil. Por ejemplo, puede limitar el tamaño de las representaciones latentes, o puede agregar ruido a las entradas y entrenar a la red para recuperar las entradas originales. Estas restricciones impiden que el autoencoder copie trivialmente las entradas directamente a las salidas, lo que lo obliga a aprender formas eficientes de representar los datos. En resumen, las codificaciones son subproductos del autocodificador que aprende la función de identidad bajo algunas restricciones.

* Los GAN se componen de dos redes neuronales: un generador que intenta generar datos que se parecen a los datos de entrenamiento, y un discriminador que trata de distinguir datos reales de datos falsos. Esta arquitectura es muy original en el aprendizaje profundo en el sentido de que el generador y el discriminador compiten entre sí durante el entrenamiento: el generador a menudo se compara con un criminal que intenta hacer dinero falso realista, mientras que el discriminador es como el investigador de la policía que intenta distinguir el dinero real de la falsificación. El entrenamiento adversario (entrenamiento de redes neuronales competidoras) es ampliamente considerada una de las innovaciones más importantes de la década de 2010. En 2016, Yann LeCun incluso dijo que era "la idea más interesante de los últimos 10 años en el aprendizaje automático".

- Un modelo probabilístico de difusión de desnuido (DDPM) está entrenado para eliminar un poco de ruido de una imagen. Si luego tomas una imagen completamente llena de ruido gaussiano y ejecutas repetidamente el modelo de difusión en esa imagen, surgirá gradualmente una imagen de alta calidad, similar a las imágenes de entrenamiento (pero no idéntica).


En este capítulo comenzaremos explorando con más profundidad cómo funcionan los autocodificadores y cómo usarlos para la reducción de la dimensionalidad, la extracción de características, el preentrenamiento no supervisado o como modelos generativos. Esto naturalmente nos llevará a los GAN. Construiremos un simple GAN para generar imágenes falsas, pero veremos que la formación suele ser bastante difícil. Discutiremos las principales dificultades que encontrará con el entrenamiento adversario, así como algunas de las principales técnicas para evitar estas dificultades. Y, por último, construiremos y entrenaremos un DDPM y lo usaremos para generar imágenes. ¡Comencemos con los autocodificadores!


# Representaciones de datos eficientes

¿Cuál de las siguientes secuencias de números te resulta más fácil de memorizar?

40, 27, 25, 36, 81, 57, 10, 73, 19, 68
50, 48, 46, 44, 42, 40, 38, 36, 34, 32, 30, 28, 26, 24, 22, 20, 18, 16, 14
A primera vista, parece que la primera secuencia debería ser más fácil, ya que es mucho más corta. Sin embargo, si miras cuidadosamente la segunda secuencia, te darás cuenta de que es solo la lista de números pares de 50 a 14. Una vez que notas este patrón, la segunda secuencia se vuelve mucho más fácil de memorizar que la primera porque solo necesitas recordar el patrón (es decir, los números pares decrecientes) y los números iniciales y finales (es decir, 50 y 14). Ten en cuenta que si pudieras memorizar rápida y fácilmente secuencias muy largas, no te importaría mucho la existencia de un patrón en la segunda secuencia. Aprenderías cada número de memoria, y eso sería todo. El hecho de que sea difícil memorizar secuencias largas es lo que hace que sea útil reconocer patrones, y espero que esto aclare por qué restringir un autocodificador durante el entrenamiento lo empuja a descubrir y explotar patrones en los datos.

La relación entre la memoria, la percepción y la coincidencia de patrones fue estudiada por William Chase y Herbert Simon⁠1 a principios de la década de 1970. Observaron que los jugadores expertos de ajedrez fueron capaces de memorizar las posiciones de todas las piezas de un juego mirando el tablero durante solo cinco segundos, una tarea que la mayoría de la gente encontraría imposible. Sin embargo, este fue solo el caso cuando las piezas se colocaban en posiciones realistas (de juegos reales), no cuando las piezas se colocaban al azar. Los expertos en ajedrez no tienen una memoria mucho mejor que tú y yo; solo ven los patrones de ajedrez más fácilmente, gracias a su experiencia con el juego. Los patrones de la nota les ayuda a almacenar información de manera eficiente.

Al igual que los jugadores de ajedrez en este experimento de memoria, un autocodificador mira las entradas, las convierte en una representación latente eficiente y luego escupe algo que (con suerte) se ve muy cerca de las entradas. Un autocodificador siempre se compone de dos partes: un codificador (o red de reconocimiento) que convierte las entradas en una representación latente, seguido de un decodificador (o red generativa) que convierte la representación interna en las salidas (ver Figura 17-1).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1701.png)

(_Figura 17-1. El experimento de memoria de ajedrez (izquierda) y un autocodificador simple (derecha)_)

Como puede ver, un autocodificador suele tener la misma arquitectura que un perceptrón multicapa (MLP; véase el capítulo 10), excepto que el número de neuronas en la capa de salida debe ser igual al número de entradas. En este ejemplo, solo hay una capa oculta compuesta por dos neuronas (el codificador) y una capa de salida compuesta por tres neuronas (el decodificador). Las salidas a menudo se llaman reconstrucciones porque el autocodificador intenta reconstruir las entradas. La función de costo contiene una pérdida de reconstrucción que penaliza el modelo cuando las reconstrucciones son diferentes de las entradas.

Debido a que la representación interna tiene una dimensionalidad más baja que los datos de entrada (es 2D en lugar de 3D), se dice que el autoencoder está incompleto. Un autocodificador incompleto no puede copiar trivialmente sus entradas a las codificaciones, sin embargo, debe encontrar una manera de generar una copia de sus entradas. Se ve obligado a aprender las características más importantes de los datos de entrada (y a dejar de lado las que no son importantes).

Veamos cómo implementar un autocodificador subcompleto muy simple para reducir la dimensionalidad.


# Realizar PCA con un autocodificador lineal incompleto

Si el autocodificador utiliza solo activaciones lineales y la función de costo es el error al cuadrado medio (MSE), entonces termina realizando un análisis de componentes principales (PCA; ver Capítulo 8).

El siguiente código construye un autocodificador lineal simple para realizar PCA en un conjunto de datos 3D, proyectándolo en 2D:

In [1]:
import tensorflow as tf

encoder = tf.keras.Sequential([tf.keras.layers.Dense(2)])
decoder = tf.keras.Sequential([tf.keras.layers.Dense(3)])
autoencoder = tf.keras.Sequential([encoder, decoder])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.5)
autoencoder.compile(loss="mse", optimizer=optimizer)

2024-03-28 02:22:12.681118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 02:22:19.242326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Este código realmente no es muy diferente de todos los MLP que construimos en capítulos anteriores, pero hay algunas cosas a tener en cuenta:

* Organizamos el codificador automático en dos subcomponentes: el codificador y el decodificador. Ambos son modelos `Sequential` regulares con una única capa Densa cada uno, y el codificador automático es un modelo secuencial que contiene el codificador seguido del decodificador (recuerde que un modelo se puede usar como capa en otro modelo).

- El número de salidas del autocodificador es igual al número de entradas (es decir, 3).

* Para realizar PCA, no utilizamos ninguna función de activación (es decir, todas las neuronas son lineales), y la función de coste es la MSE. Eso se debe a que la PCA es una transformación lineal. En breve veremos autocodificadores más complejos y no lineales.

Ahora entrenemos el modelo en el mismo conjunto de datos 3D generado simple que usamos en el capítulo 8 y usémoslo para codificar ese conjunto de datos (es decir, proyectarlo en 2D):

In [ ]:
X_train = [...]  # generate a 3D dataset, like in Chapter 8
history = autoencoder.fit(X_train, X_train, epochs=500, verbose=False)
codings = encoder.predict(X_train)

Tenga en cuenta que `X_train` se utiliza tanto como entradas como como objetivos. La Figura 17-2 muestra el conjunto de datos 3D original (a la izquierda) y la salida de la capa oculta del codificador automático (es decir, la capa de codificación, a la derecha). Como puede ver, el codificador automático encontró el mejor plano 2D para proyectar los datos, preservando tanta variación en los datos como pudo (al igual que PCA).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1702.png)

(_Figura 17-2. PCA aproximada realizada por un autocodificador lineal incompleto_)


#### NOTA

Puedes pensar en un autocodificador como una forma de aprendizaje autosupervisado, ya que se basa en una técnica de aprendizaje supervisado con etiquetas generadas automáticamente (en este caso simplemente iguales a las entradas).

#### -----------------------------------------------------------------------------------


# Autocodificadores apilados

Al igual que otras redes neuronales que hemos discutido, los autocodificadores pueden tener múltiples capas ocultas. En este caso, se les llama autocodificadores apilados (o autocodificadores profundos). Añadir más capas ayuda al autocodificador a aprender codificaciones más complejas. Dicho esto, hay que tener cuidado de no hacer que el autoencoder sea demasiado potente. Imagina un codificador tan potente que solo aprende a asignar cada entrada a un solo número arbitrario (y el decodificador aprende el mapeo inverso). Obviamente, un autocodificador de este tipo reconstruirá los datos de entrenamiento perfectamente, pero no habrá aprendido ninguna representación de datos útil en el proceso, y es poco probable que se generalice bien a nuevas instancias.

La arquitectura de un autocodificador apilado suele ser simétrica con respecto a la capa oculta central (la capa de codificación). En pocas palabras, parece un sándwich. Por ejemplo, un autocodificador para Fashion MNIST (introducido en el capítulo 10) puede tener 784 entradas, seguida de una capa oculta con 100 neuronas, luego una capa central oculta de 30 neuronas, luego otra capa oculta con 100 neuronas y una capa de salida con 784 neuronas. Este autocodificador apilado se representa en la Figura 17-3.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1703.png)

(_Figura 17-3. Autocodificador apilado_)


## Implementación de un autocodificador apilado usando Keras

Puedes implementar un autoencoder apilado de manera muy similar a un MLP profundo normal:

In [ ]:
stacked_encoder = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(30, activation="relu"),
])
stacked_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(28 * 28),
    tf.keras.layers.Reshape([28, 28])
])
stacked_ae = tf.keras.Sequential([stacked_encoder, stacked_decoder])

stacked_ae.compile(loss="mse", optimizer="nadam")
history = stacked_ae.fit(X_train, X_train, epochs=20,
                         validation_data=(X_valid, X_valid))

Vamos a revisar este código:

- Al igual que antes, dividimos el modelo de autocodificador en dos submodelos: el codificador y el decodificador.

* El codificador toma imágenes en escala de grises de 28 × 28 píxeles, las aplana para que cada imagen se represente como un vector de tamaño 784, luego procesa estos vectores a través de dos capas `Dense` de tamaños decrecientes (100 unidades y luego 30 unidades), ambas usando ReLU. función de activación. Para cada imagen de entrada, el codificador genera un vector de tamaño 30.

- El decodificador toma codificaciones de tamaño 30 (salida por el codificador) y las procesa a través de dos capas `Dense` de tamaños crecientes (100 unidades y luego 784 unidades), y reforma los vectores finales en matrices de 28 × 28 para que las salidas del decodificador tengan la misma forma que las entradas del codificador.

* Al compilar el autoencoder apilado, utilizamos la pérdida de MSE y la optimización de Nadam.

Finalmente, entrenamos el modelo usando `X_train` como entradas y objetivos. De manera similar, usamos `X_valid` como entradas y objetivos de validación.


## Visualización de las reconstrucciones

Una forma de asegurarse de que un autocodificador esté debidamente entrenado es comparar las entradas y las salidas: las diferencias no deben ser demasiado significativas. Vamos a trazar algunas imágenes del conjunto de validación, así como sus reconstrucciones:

In [ ]:
import numpy as np

def plot_reconstructions(model, images=X_valid, n_images=5):
    reconstructions = np.clip(model.predict(images[:n_images]), 0, 1)
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plt.imshow(images[image_index], cmap="binary")
        plt.axis("off")
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plt.imshow(reconstructions[image_index], cmap="binary")
        plt.axis("off")

plot_reconstructions(stacked_ae)
plt.show()

La figura 17-4 muestra las imágenes resultantes.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1704.png)

(Figura 17-4. Imágenes originales (arriba) y sus reconstrucciones (abajo))

Las reconstrucciones son reconocibles, pero con demasiado pérdidas. Es posible que necesitemos entrenar el modelo durante más tiempo, o hacer que el codificador y el decodificador sean más profundos, o hacer que las codificaciones sean más grandes. Pero si hacemos que la red sea demasiado poderosa, logrará hacer reconstrucciones perfectas sin haber aprendido ningún patrón útil en los datos. Por ahora, vamos con este modelo.


## Visualización del conjunto de datos de Fashion MNIST


Ahora que hemos entrenado un autocodificador apilado, podemos usarlo para reducir la dimensionalidad del conjunto de datos. Para la visualización, esto no da grandes resultados en comparación con otros algoritmos de reducción de la dimensionalidad (como los que discutimos en el capítulo 8), pero una gran ventaja de los autocodificadores es que pueden manejar grandes conjuntos de datos con muchas instancias y muchas características. Por lo tanto, una estrategia es usar un autocodificador para reducir la dimensionalidad a un nivel razonable, y luego usar otro algoritmo de reducción de dimensionalidad para la visualización. Usemos esta estrategia para visualizar Fashion MNIST. Primero usaremos el codificador de nuestro autocodificador apilado para reducir la dimensionalidad a 30, luego usaremos la implementación de Scikit-Learn del algoritmo t-SNE para reducir la dimensionalidad a 2 para la visualización:

In [ ]:
from sklearn.manifold import TSNE

X_valid_compressed = stacked_encoder.predict(X_valid)
tsne = TSNE(init="pca", learning_rate="auto", random_state=42)
X_valid_2D = tsne.fit_transform(X_valid_compressed)

Ahora podemos trazar el conjunto de datos:

In [ ]:
plt.scatter(X_valid_2D[:, 0], X_valid_2D[:, 1], c=y_valid, s=10, cmap="tab10")
plt.show()

La figura 17-5 muestra el gráfico de dispersión resultante, embellecido un poco al mostrar algunas de las imágenes. El algoritmo t-SNE identificó varios grupos que coinciden razonablemente bien con las clases (cada clase está representada por un color diferente).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1705.png)

(_Figura 17-5. Visualización de moda de MNIST utilizando un autoencoder seguido de t-SNE_)

Por lo tanto, los autocodificadores se pueden utilizar para reducir la dimensionalidad. Otra solicitud es para el preentrenamiento no supervisado.


## Preentrenamiento No Supervisado utilizando Autocodificadores Apilados


Como discutimos en el capítulo 11, si está abordando una tarea supervisada compleja pero no tiene muchos datos de entrenamiento etiquetados, una solución es encontrar una red neuronal que realice una tarea similar y reutilizar sus capas inferiores. Esto hace posible entrenar un modelo de alto rendimiento utilizando pocos datos de entrenamiento porque su red neuronal no tendrá que aprender todas las características de bajo nivel; solo reutilizará los detectores de características aprendidos por la red existente.

Del mismo modo, si tiene un conjunto de datos grande pero la mayor parte no está etiquetada, primero puede entrenar a un autocodificador apilado usando todos los datos, luego reutilizar las capas inferiores para crear una red neuronal para su tarea real y entrenarla utilizando los datos etiquetados. Por ejemplo, la Figura 17-6 muestra cómo usar un autocodificador apilado para realizar un preentrenamiento no supervisado para una red neuronal de clasificación. Al entrenar al clasificador, si realmente no tiene muchos datos de entrenamiento etiquetados, es posible que desee congelar las capas preentrenadas (al menos las inferiores).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1706.png)

(_Figura 17-6. Preentrenamiento no supervisado con autocodificadores_)


#### NOTA

Es común tener muchos datos sin etiquetar y pocos datos etiquetados. Construir un gran conjunto de datos sin etiquetar suele ser barato (por ejemplo, un simple guión puede descargar millones de imágenes de Internet), pero etiquetar esas imágenes (por ejemplo, clasificarlas como lindas o no) generalmente solo los humanos pueden hacer de manera confiable. El etiquetado de instancias lleva mucho tiempo y es costoso, por lo que es normal tener solo unos pocos miles de instancias etiquetadas por humanos, o incluso menos.

#### ------------------------------------------------------------------------------------

No hay nada especial en la implementación: simplemente entrene a un autocodificador utilizando todos los datos de entrenamiento (etiquetado más sin etiquetar), luego reutilice sus capas de codificador para crear una nueva red neuronal (consulte los ejercicios al final de este capítulo para ver un ejemplo).

A continuación, echemos un vistazo a algunas técnicas para entrenar a los autocodificadores apilados.


### Atar pesas

Cuando un autocodificador es cuidadosamente simétrico, como el que acabamos de construir, una técnica común es atar los pesos de las capas del decodificador a los pesos de las capas del codificador. Esto reduce a la mitad el número de pesos en el modelo, acelerando el entrenamiento y limitando el riesgo de sobreajuste. Específicamente, si el autoencodificador tiene un total de N capas (sin contar la capa de entrada), y WL representa los pesos de conexión de la capa Lth (por ejemplo, la capa 1 es la primera capa oculta, la capa N/2 es la capa de codificación y la capa N es la capa de salida), entonces los pesos de la capa del decodificador se pueden definir como WL = WN-L+1⊺ (con L = N / 2 + 1, ... , N).

Para atar pesos entre capas usando Keras, definamos una capa personalizada:

In [ ]:
class DenseTranspose(tf.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.dense = dense
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias",
                                      shape=self.dense.input_shape[-1],
                                      initializer="zeros")
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)
        return self.activation(Z + self.biases)

Esta capa personalizada actúa como una capa Densa normal, pero utiliza los pesos de otra capa Densa, transpuesta (establecer `transpose_b=True` es equivalente a transponer el segundo argumento, pero es más eficiente ya que realiza la transposición sobre la marcha dentro de la operación `matmul()` ). Sin embargo, utiliza su propio vector de sesgo. Ahora podemos construir un nuevo codificador automático apilado, muy parecido al anterior pero con las capas Densa del decodificador unidas a las capas Densa del codificador:

In [ ]:
dense_1 = tf.keras.layers.Dense(100, activation="relu")
dense_2 = tf.keras.layers.Dense(30, activation="relu")

tied_encoder = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    dense_1,
    dense_2
])

tied_decoder = tf.keras.Sequential([
    DenseTranspose(dense_2, activation="relu"),
    DenseTranspose(dense_1),
    tf.keras.layers.Reshape([28, 28])
])

tied_ae = tf.keras.Sequential([tied_encoder, tied_decoder])

Este modelo logra aproximadamente el mismo error de reconstrucción que el modelo anterior, utilizando casi la mitad del número de parámetros.


## Entrenando a un autocodificador a la vez


En lugar de entrenar a todo el autocodificador apilado de una sola vez como acabamos de hacer, es posible entrenar un autocodificador poco profundo a la vez, y luego apilarlos todos en un solo autocodificador apilado (de ahí el nombre), como se muestra en la Figura 17-7. Esta técnica no se usa tanto en estos días, pero aún así puedes encontrarte con artículos que hablan de "entrenamiento en capas codicioso", por lo que es bueno saber lo que significa.

(_https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1707.png_)


Durante la primera fase de entrenamiento, el primer autoencoder aprende a reconstruir las entradas. Luego codificamos todo el conjunto de entrenamiento usando este primer autocodificador, y esto nos da un nuevo conjunto de entrenamiento (comprimido). Luego entrenamos a un segundo autocodificador en este nuevo conjunto de datos. Esta es la segunda fase de entrenamiento. Finalmente, construimos un gran sándwich usando todos estos autocodificadores, como se muestra en la Figura 17-7 (es decir, primero apilamos las capas ocultas de cada autocodificador, luego las capas de salida en orden inverso). Esto nos da el autoencodificador apilado final (consulte la sección "Entrenar a un autoencoder a la vez" en el cuaderno del capítulo para una implementación). Podríamos entrenar fácilmente más autocodificadores de esta manera, construyendo un autocodificador apilado muy profundo.

Como mencioné anteriormente, uno de los desencadenantes del tsunami de aprendizaje profundo fue el descubrimiento en 2006 por Geoffrey Hinton et al. de que las redes neuronales profundas se pueden preentrenar de manera no supervisada, utilizando este enfoque codicioso en capas. Usaron máquinas Boltzmann restringidas (RBM; ver https://homl.info/extra-anns) para este propósito, pero en 2007 Yoshia Bengio et al. mostraron que los autocodificadores funcionaban igual. Durante varios años, esta fue la única forma eficiente de entrenar redes profundas, hasta que muchas de las técnicas introducidas en el capítulo 11 hicieron posible entrenar una red profunda de una sola vez.

Los autocodificadores no se limitan a redes densas: también puedes construir autocodificadores convolucionales. Echemos un vistazo a esto ahora.


# Autocodificadores convolucionales


Si se trata de imágenes, entonces los autocodificadores que hemos visto hasta ahora no funcionarán bien (a menos que las imágenes sean muy pequeñas): como viste en el capítulo 14, las redes neuronales convolucionales son mucho más adecuadas que las redes densas para trabajar con imágenes. Por lo tanto, si desea construir un autocodificador para imágenes (por ejemplo, para el preentrenamiento no supervisado o la reducción de la dimensionalidad), tendrá que construir un autocodificador convolucional.⁠ El codificador es un CNN regular compuesto de capas convolucionales y capas de agrupación. Por lo general, reduce la dimensionalidad espacial de las entradas (es decir, la altura y el ancho) al tiempo que aumenta la profundidad (es decir, el número de mapas de características). El decodificador debe hacer lo contrario (redimensionar la imagen y reducir su profundidad a las dimensiones originales), y para esto puede usar capas convolucionales transpuestas (alternativamente, podría combinar capas de muestreo superior con capas convolucionales). Aquí hay un autocodificador convolucional básico para Fashion MNIST:

In [ ]:
conv_encoder = tf.keras.Sequential([
    tf.keras.layers.Reshape([28, 28, 1]),
    tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 14 × 14 x 16
    tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 7 × 7 x 32
    tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 3 × 3 x 64
    tf.keras.layers.Conv2D(30, 3, padding="same", activation="relu"),
    tf.keras.layers.GlobalAvgPool2D()  # output: 30
])
conv_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(3 * 3 * 16),
    tf.keras.layers.Reshape((3, 3, 16)),
    tf.keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu"),
    tf.keras.layers.Conv2DTranspose(16, 3, strides=2, padding="same",
                                    activation="relu"),
    tf.keras.layers.Conv2DTranspose(1, 3, strides=2, padding="same"),
    tf.keras.layers.Reshape([28, 28])
])
conv_ae = tf.keras.Sequential([conv_encoder, conv_decoder])

También es posible crear autocodificadores con otros tipos de arquitectura, como RNN (ver el cuaderno para ver un ejemplo).

Vale, demos un paso atrás por un segundo. Hasta ahora hemos analizado varios tipos de autocodificadores (básicos, apilados y convolucionales), y cómo entrenarlos (ya sea en una toma o capa por capa). También ansentamos un par de aplicaciones: visualización de datos y preentrenamiento no supervisado.

Hasta ahora, con el fin de obligar al autoencoder a aprender características interesantes, hemos limitado el tamaño de la capa de codificación, por lo que no está incompleta. En realidad, hay muchos otros tipos de restricciones que se pueden utilizar, incluidas las que permiten que la capa de codificación sea tan grande como las entradas, o incluso más grande, lo que resulta en un autocodificador sobrecompleto. Luego, en las siguientes secciones veremos algunos tipos más de autocodificadores: autocodificadores de desarido, autocodificadores dispersos y autocodificadores de variación.


# Autocodificadores de desardenado


Otra forma de obligar al autocodificador a aprender características útiles es añadir ruido a sus entradas, entrenándolo para recuperar las entradas originales y libres de ruido. Esta idea ha existido desde la década de 1980 (por ejemplo, se menciona en la tesis de maestría de Yann LeCun de 1987). En un artículo de 2008, Pascal Vincent et al. mostraron que los autocodificadores también podrían usarse para la extracción de características. En un artículo de 2010, Vincent et al. introdujeron autocodificadores de desnado apilados.

El ruido puede ser ruido gaussiano puro añadido a las entradas, o puede ser entradas apagadas al azar, al igual que en la caída (introducida en el capítulo 11). La figura 17-8 muestra ambas opciones.

La implementación es sencilla: es un codificador automático apilado normal con una capa adicional `Dropout` aplicada a las entradas del codificador (o podrías usar una capa "GaussianNoise" en su lugar). Recuerde que la capa `Dropout` solo está activa durante el entrenamiento (al igual que la capa `GaussianNoise`):

In [ ]:
dropout_encoder = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(30, activation="relu")
])
dropout_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(28 * 28),
    tf.keras.layers.Reshape([28, 28])
])
dropout_ae = tf.keras.Sequential([dropout_encoder, dropout_decoder])

1[](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1708.png)

(_Figura 17-8. Autocodificadores de ruido, con ruido gaussiano (izquierda) o abandono (derecha)_)

La figura 17-9 muestra algunas imágenes ruidosas (con la mitad de los píxeles desactivados) y las imágenes reconstruidas por el autocodificador de desnuido basado en la caída. Observe cómo el autocodificador adivina los detalles que en realidad no están en la entrada, como la parte superior de la camisa blanca (fila inferior, cuarta imagen). Como puede ver, los autocodificadores de denodo no solo se pueden usar para la visualización de datos o el preentrenamiento no supervisado, como los otros autocodificadores que hemos discutido hasta ahora, sino que también se pueden usar de manera muy simple y eficiente para eliminar el ruido de las imágenes.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1709.png)

(_Figura 17-9. Imágenes ruidosas (arriba) y sus reconstrucciones (abajo)_)


# Autocodificadores escasos


Otro tipo de restricción que a menudo conduce a una buena extracción de características es la esparidad: al agregar un término apropiado a la función de costo, el autocodificador se presiona para reducir el número de neuronas activas en la capa de codificación. Por ejemplo, puede ser empujado a tener en promedio solo un 5 % de neuronas significativamente activas en la capa de codificación. Esto obliga al autoencoder a representar cada entrada como una combinación de un pequeño número de activaciones. Como resultado, cada neurona en la capa de codificación generalmente termina representando una característica útil (si pudieras hablar solo unas pocas palabras al mes, probablemente tratarías de hacer que valgan la pena escucharlas).

Un enfoque simple es usar la función de activación sigmoide en la capa de codificación (para restringir las codificaciones a valores entre 0 y 1), usar una capa de codificación grande (por ejemplo, con 300 unidades) y agregar algo de regularización ℓ1 a las activaciones de la capa de codificación. El decodificador es solo un decodificador normal:

In [ ]:
sparse_l1_encoder = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(300, activation="sigmoid"),
    tf.keras.layers.ActivityRegularization(l1=1e-4)
])
sparse_l1_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(28 * 28),
    tf.keras.layers.Reshape([28, 28])
])
sparse_l1_ae = tf.keras.Sequential([sparse_l1_encoder, sparse_l1_decoder])

Esta capa `ActivityRegularization` simplemente devuelve sus entradas, pero como efecto secundario agrega una pérdida de entrenamiento igual a la suma de los valores absolutos de sus entradas. Esto sólo afecta al entrenamiento. De manera equivalente, puede eliminar la capa `ActivityRegularization` y configurar `Activity_regularizer=tf.keras.regularizers.l1(1e-4)` en la capa anterior. Esta penalización alentará a la red neuronal a producir codificaciones cercanas a 0, pero como también será penalizada si no reconstruye las entradas correctamente, tendrá que generar al menos algunos valores distintos de cero. El uso de la norma ℓ1 en lugar de la norma ℓ2 impulsará a la red neuronal a preservar las codificaciones más importantes y al mismo tiempo eliminar las que no son necesarias para la imagen de entrada (en lugar de simplemente reducir todas las codificaciones).

Otro enfoque, que a menudo produce mejores resultados, es medir la esparidad real de la capa de codificación en cada iteración de entrenamiento, y penalizar el modelo cuando la esparidad medida difiere de la esparsidad objetivo. Lo hacemos calculando la activación promedio de cada neurona en la capa de codificación, a lo largo de todo el lote de entrenamiento. El tamaño del lote no debe ser demasiado pequeño, de lo contrario la media no será precisa.

Una vez que tenemos la activación media por neurona, queremos penalizar a las neuronas que están demasiado activas, o no lo suficientemente activas, añadiendo una pérdida de esparidad a la función de costo. Por ejemplo, si medimos que una neurona tiene una activación promedio de 0,3, pero la esparidad objetivo es 0,1, debe ser penalizada para activar menos. Un enfoque podría ser simplemente agregar el error cuadrado (0,3 - 0,1)2 a la función de costo, pero en la práctica un mejor enfoque es usar la divergencia Kullback-Leibler (KL) (discutida brevemente en el Capítulo 4), que tiene gradientes mucho más fuertes que el error cuadrado medio, como se puede ver en la Figura 17-10.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1710.png)

(_Figura 17-10. La pérdida de la esparidad_)

Dadas dos distribuciones de probabilidad discretas P y Q, la divergencia de KL entre estas distribuciones, señalada DKL(P ∥ Q), se puede calcular utilizando la ecuación 17-1.

### Ecuación 17-1. Divergencia Kullback-Leibler

<a href="https://imgbb.com/"><img src="https://i.ibb.co/ZXT2gNN/Captura-de-pantalla-2024-03-28-a-las-2-44-54.png" alt="Captura-de-pantalla-2024-03-28-a-las-2-44-54" border="0"></a>

En nuestro caso, queremos medir la divergencia entre la probabilidad objetivo p de que una neurona en la capa de codificación se active y la probabilidad real q, estimada midiendo la activación media en el lote de entrenamiento. Por lo tanto, la divergencia KL se simplifica a la ecuación 17-2.


### Ecuación 17-2. Divergencia de KL entre la esparidad objetivo p y la esparsidad real q


<a href="https://imgbb.com/"><img src="https://i.ibb.co/MGwWt7X/Captura-de-pantalla-2024-03-28-a-las-2-45-59.png" alt="Captura-de-pantalla-2024-03-28-a-las-2-45-59" border="0"></a>

Una vez que hemos calculado la pérdida de esparidad para cada neurona en la capa de codificación, sumamos estas pérdidas y añadimos el resultado a la función de costo. Con el fin de controlar la importancia relativa de la pérdida de esparidad y la pérdida de reconstrucción, podemos multiplicar la pérdida de esparidad por un hiperparámetro de peso de la esparidad. Si este peso es demasiado alto, el modelo se aderará estrechamente a la escaridad objetivo, pero puede no reconstruir las entradas correctamente, lo que hará que el modelo sea inútil. Por el contrario, si es demasiado bajo, el modelo ignorará en su mayoría el objetivo de la esparidad y no aprenderá ninguna característica interesante.

Ahora tenemos todo lo que necesitamos para implementar un autocodificador disperso basado en la divergencia de KL. Primero, vamos a crear un regularizador personalizado para aplicar la regularización de la divergencia de KL:

In [ ]:
kl_divergence = tf.keras.losses.kullback_leibler_divergence

class KLDivergenceRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, weight, target):
        self.weight = weight
        self.target = target

    def __call__(self, inputs):
        mean_activities = tf.reduce_mean(inputs, axis=0)
        return self.weight * (
            kl_divergence(self.target, mean_activities) +
            kl_divergence(1. - self.target, 1. - mean_activities))

Ahora podemos construir el codificador automático disperso, usando `KLDivergenceRegularizer` para las activaciones de la capa de codificación:

In [ ]:
kld_reg = KLDivergenceRegularizer(weight=5e-3, target=0.1)
sparse_kl_encoder = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(300, activation="sigmoid",
                          activity_regularizer=kld_reg)
])
sparse_kl_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(28 * 28),
    tf.keras.layers.Reshape([28, 28])
])
sparse_kl_ae = tf.keras.Sequential([sparse_kl_encoder, sparse_kl_decoder])

Después de entrenar a este codificador automático disperso en Fashion MNIST, la capa de codificación tendrá aproximadamente un 10% de escasez.

Autocodificadores variacionales

Una importante categoría de autocodificadores fue introducida en 2013 por Diederik Kingma y Max Welling⁠6 y rápidamente se convirtió en una de las variantes más populares: los autocodificadores variacionales (VAE).

Los VAE son bastante diferentes de todos los autocodificadores que hemos discutido hasta ahora, de estas maneras particulares:

* Son autocodificadores probabilísticos, lo que significa que sus salidas están determinadas en parte por el azar, incluso después del entrenamiento (a diferencia de los autocodificadores de desarrudo, que utilizan la aleatoriedad solo durante el entrenamiento).

- Lo más importante es que son autocodificadores generativos, lo que significa que pueden generar nuevas instancias que parecen que fueron muestreadas del conjunto de entrenamiento.

Ambas propiedades hacen que los VAE sean bastante similares a los RBM, pero son más fáciles de entrenar, y el proceso de muestreo es mucho más rápido (con los RBM es necesario esperar a que la red se estabilice en un "equilibrio térmico" antes de poder muestrear una nueva instancia). Como su nombre indica, los autocodificadores variacionales realizan una inferencia bayesiana variacional, que es una forma eficiente de llevar a cabo una inferencia bayesiana aproximada. Recuerde que la inferencia bayesiana significa actualizar una distribución de probabilidad basada en nuevos datos, utilizando ecuaciones derivadas del teorema de Bayes. La distribución original se llama anterior, mientras que la distribución actualizada se llama posterior. En nuestro caso, queremos encontrar una buena aproximación de la distribución de datos. Una vez que tengamos eso, podemos probarlo.

Echemos un vistazo a cómo funcionan los VAE. La figura 17-11 (izquierda) muestra un autocodificador variacional. Puede reconocer la estructura básica de todos los autocodificadores, con un codificador seguido de un decodificador (en este ejemplo, ambos tienen dos capas ocultas), pero hay un giro: en lugar de producir directamente una codificación para una entrada dada, el codificador produce una codificación media μ y una desviación estándar σ. La codificación real se muestrea al azar a partir de una distribución gaussiana con media μ y desviación estándar σ. Después de eso, el decodificador decodifica la codificación muestreada normalmente. La parte derecha del diagrama muestra una instancia de entrenamiento que pasa por este autocodificador. En primer lugar, el codificador produce μ y σ, luego se muestrea una codificación al azar (tenga en cuenta que no se encuentra exactamente en μ), y finalmente esta codificación se decodifica; la salida final se asemeja a la instancia de entrenamiento.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1711.png)

Como se puede ver en el diagrama, aunque las entradas pueden tener una distribución muy complicada, un autocodificador variacional tiende a producir codificaciones que parecen haber sido muestreadas de una distribución gaussiana simple: ⁠7 durante el entrenamiento, la función de costo (discutida a continuación) empuja las codificaciones a migrar gradualmente dentro del espacio de codificación (también llamado el espacio latente) para terminar pareciendo una nube de puntos gausss. Una gran consecuencia es que después de entrenar a un autocodificador variacional, puedes generar muy fácilmente una nueva instancia: ¡solo tienes que probar una codificación aleatoria de la distribución gaussiana, descifrarla y voilà!

Ahora, echemos un vistazo a la función de coste. Se compone de dos partes. La primera es la pérdida de reconstrucción habitual que empuja al autocodificador a reproducir sus entradas. Podemos usar el MSE para esto, como lo hicimos antes. La segunda es la pérdida latente que empuja al autoencoder a tener codificaciones que parecen haber sido muestreadas de una distribución gaussiana simple: es la divergencia KL entre la distribución objetivo (es decir, la distribución gaussiana) y la distribución real de las codificaciones. Las matemáticas son un poco más complejas que con el autocodificador disperso, en particular debido al ruido gaussiano, que limita la cantidad de información que se puede transmitir a la capa de codificación. Esto empuja al autoencoder a aprender características útiles. Afortunadamente, las ecuaciones se simplifican, por lo que la pérdida latente se puede calcular usando la ecuación 17-3.⁠8

### Ecuación 17-3. La pérdida latente del autocodificador variacional

<a href="https://imgbb.com/"><img src="https://i.ibb.co/0fsczvZ/Captura-de-pantalla-2024-03-30-a-las-5-32-11.png" alt="Captura-de-pantalla-2024-03-30-a-las-5-32-11" border="0"></a><br /><a target='_blank' href='https://es.imgbb.com/'></a><br />

En esta ecuación, ℒ es la pérdida latente, n es la dimensionalidad de las codificaciones, y μi y σi son la media y la desviación estándar del componente ith de las codificaciones. Los vectores μ y σ (que contienen todos los μi y σi) son emitidos por el codificador, como se muestra en la Figura 17-11 (izquierda).

Un ajuste común en la arquitectura del autocodificador variacional es hacer que la salida del codificador sea γ = log(σ2) en lugar de σ. La pérdida latente se puede calcular como se muestra en la ecuación 17-4. Este enfoque es numéricamente más estable y acelera el entrenamiento.

### Ecuación 17-4. La pérdida latente del autocodificador variacional, reescrita usando γ = log(σ²)

<a href="https://imgbb.com/"><img src="https://i.ibb.co/B37tfcb/Captura-de-pantalla-2024-03-30-a-las-5-35-40.png" alt="Captura-de-pantalla-2024-03-30-a-las-5-35-40" border="0"></a>

Comencemos a construir un autocodificador variacional para Fashion MNIST (como se muestra en la Figura 17-11, pero usando el ajuste γ). En primer lugar, necesitaremos una capa personalizada para muestrear las codificaciones, dadas μ y γ:

In [ ]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return tf.random.normal(tf.shape(log_var)) * tf.exp(log_var / 2) + mean

Esta capa de muestreo toma dos entradas: `mean` (μ) y `log_var` (γ). Utiliza la función `tf.random.normal()` para muestrear un vector aleatorio (de la misma forma que γ) de la distribución gaussiana, con media 0 y desviación estándar 1. Luego lo multiplica por exp(γ / 2) (que es igual a σ, como puedes comprobar matemáticamente), y finalmente suma μ y devuelve el resultado. Esto muestra un vector de codificación de la distribución gaussiana con media μ y desviación estándar σ.

A continuación, podemos crear el codificador, utilizando la API funcional porque el modelo no es completamente secuencial:

In [ ]:
codings_size = 10

inputs = tf.keras.layers.Input(shape=[28, 28])
Z = tf.keras.layers.Flatten()(inputs)
Z = tf.keras.layers.Dense(150, activation="relu")(Z)
Z = tf.keras.layers.Dense(100, activation="relu")(Z)
codings_mean = tf.keras.layers.Dense(codings_size)(Z)  # μ
codings_log_var = tf.keras.layers.Dense(codings_size)(Z)  # γ
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = tf.keras.Model(
    inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

Tenga en cuenta que las capas `Dense` que generan `codings_mean` (μ) y `codings_log_var` (γ) tienen las mismas entradas (es decir, las salidas de la segunda capa densa). Luego pasamos tanto codings_mean como codings_log_var a la capa de muestreo. Finalmente, el modelo `variacional_encoder` tiene tres salidas. Solo se requieren las `codings`, pero también agregamos `codings_mean` y `codings_log_var`, en caso de que queramos inspeccionar sus valores. Ahora construyamos el decodificador:

In [ ]:
decoder_inputs = tf.keras.layers.Input(shape=[codings_size])
x = tf.keras.layers.Dense(100, activation="relu")(decoder_inputs)
x = tf.keras.layers.Dense(150, activation="relu")(x)
x = tf.keras.layers.Dense(28 * 28)(x)
outputs = tf.keras.layers.Reshape([28, 28])(x)
variational_decoder = tf.keras.Model(inputs=[decoder_inputs], outputs=[outputs])

Para este decodificador, podríamos haber utilizado la API secuencial en lugar de la API funcional, ya que en realidad es solo una simple pila de capas, prácticamente idénticas a muchos de los decodificadores que hemos construido hasta ahora. Por último, construyamos el modelo de autocodificador variacional:

In [ ]:
_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_ae = tf.keras.Model(inputs=[inputs], outputs=[reconstructions])

Ignoramos las dos primeras salidas del codificador (solo queremos alimentar las codificaciones al decodificador). Por último, debemos añadir la pérdida latente y la pérdida de reconstrucción:

In [ ]:
latent_loss = -0.5 * tf.reduce_sum(
    1 + codings_log_var - tf.exp(codings_log_var) - tf.square(codings_mean),
    axis=-1)
variational_ae.add_loss(tf.reduce_mean(latent_loss) / 784.)

Primero aplicamos la ecuación 17-4 para calcular la pérdida latente para cada instancia del lote, sumando el último eje. Luego calculamos la pérdida media de todas las instancias del lote y dividimos el resultado por 784 para asegurarnos de que tenga la escala adecuada en comparación con la pérdida de reconstrucción. De hecho, se supone que la pérdida de reconstrucción del codificador automático variacional es la suma de los errores de reconstrucción de píxeles, pero cuando Keras calcula la pérdida `"mse"`, calcula la media de los 784 píxeles, en lugar de la suma. Entonces, la pérdida de reconstrucción es 784 veces menor de lo que necesitamos. Podríamos definir una pérdida personalizada para calcular la suma en lugar de la media, pero es más sencillo dividir la pérdida latente entre 784 (la pérdida final será 784 veces menor de lo que debería ser, pero esto solo significa que debemos usar una pérdida mayor). tasa de aprendizaje).

Y finalmente, ¡podemos compilar y ajustar el autocodificador!

In [ ]:
variational_ae.compile(loss="mse", optimizer="nadam")
history = variational_ae.fit(X_train, X_train, epochs=25, batch_size=128,
                             validation_data=(X_valid, X_valid))

# Generación de imágenes de moda MNIST


Ahora usemos este autocodificador variacional para generar imágenes que parecen artículos de moda. Todo lo que tenemos que hacer es probar codificaciones aleatorias de una distribución gaussiana y decodificarlas:

In [ ]:
codings = tf.random.normal(shape=[3 * 7, codings_size])
images = variational_decoder(codings).numpy()

La figura 17-12 muestra las 12 imágenes generadas.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1712.png)

(_Figura 17-12. Imágenes de moda MNIST generadas por el autocodificador variacional_)

La mayoría de estas imágenes parecen bastante convincentes, aunque un poco demasiado borrosas. El resto no es genial, pero no seas demasiado duro con el autocodificador, ¡solo tenía unos minutos para aprender!

Los autocodificadores variacionales hacen posible realizar la interpolación semántica: en lugar de interpolar entre dos imágenes a nivel de píxel, que parecería que las dos imágenes estuvieran superperadas, podemos interpolar a nivel de codificaciones. Por ejemplo, tomemos algunas codificaciones a lo largo de una línea arbitraria en el espacio latente y descodécrelas. Tenemos una secuencia de imágenes que gradualmente van de los pantalones a los suéteres (ver Figura 17-13):

In [ ]:
codings = np.zeros([7, codings_size])
codings[:, 3] = np.linspace(-0.8, 0.8, 7)  # axis 3 looks best in this case
images = variational_decoder(codings).numpy()

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1713.png)

(_Figura 17-13. Interpolación semántica_)

Ahora dirigamos nuestra atención a los GAN: son más difíciles de entrenar, pero cuando te las arreglas para que funcionen, producen imágenes bastante increíbles.


# Redes adversarias generativas


Las redes adversarias generativas fueron propuestas en un documento de 2014⁠ de Ian Goodfellow et al., y aunque la idea emocionó a los investigadores casi al instante, tomó unos años superar algunas de las dificultades de entrenar a los GAN. Al igual que muchas grandes ideas, parece simple en retrospectiva: hacer que las redes neuronales compitan entre sí con la esperanza de que esta competencia las empuje a sobresalir. Como se muestra en la Figura 17-14, un GAN se compone de dos redes neuronales:

**generador**

    Toma una distribución aleatoria como entrada (típicamente gaussiana) y genera algunos datos, normalmente, una imagen. Puedes pensar en las entradas aleatorias como las representaciones latentes (es decir, codificaciones) de la imagen que se va a generar. Por lo tanto, como puede ver, el generador ofrece la misma funcionalidad que un decodificador en un autocodificador variacional, y se puede utilizar de la misma manera para generar nuevas imágenes: simplemente alimente un poco de ruido gausssiano y emite una imagen completamente nueva. Sin embargo, se entrena de manera muy diferente, como verás pronto.

**Discriminador**

    Toma una imagen falsa del generador o una imagen real del conjunto de entrenamiento como entrada, y debe adivinar si la imagen de entrada es falsa o real.
    
    
![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1714.png)

(_Figura 17-14. Una red adversaria generativa_)

Durante el entrenamiento, el generador y el discriminador tienen objetivos opuestos: el discriminador trata de distinguir imágenes falsas de imágenes reales, mientras que el generador trata de producir imágenes que parecen lo suficientemente reales como para engañar al discriminador. Debido a que el GAN se compone de dos redes con objetivos diferentes, no se puede entrenar como una red neuronal normal. Cada iteración de entrenamiento se divide en dos fases:

* En la primera fase, entrenamos al discriminador. Se muestrea un lote de imágenes reales del conjunto de entrenamiento y se completa con un número igual de imágenes falsas producidas por el generador. Las etiquetas se establecen en 0 para imágenes falsas y 1 para imágenes reales, y el discriminador se entrena en este lote etiquetado durante un paso, utilizando la pérdida de entropía cruzada binaria. Es importante destacar que la repropagación solo optimiza los pesos del discriminador durante esta fase.

- En la segunda fase, entrenamos al generador. Primero lo usamos para producir otro lote de imágenes falsas, y una vez más se utiliza el discriminador para decir si las imágenes son falsas o reales. Esta vez no añadimos imágenes reales en el lote, y todas las etiquetas están configuradas en 1 (real): en otras palabras, ¡queremos que el generador produzca imágenes que el discriminador (erróneamente) cree que son reales! Fundamentalmente, los pesos del discriminador se congelan durante este paso, por lo que la contrapropagación solo afecta a los pesos del generador.

#### NOTA

En realidad, el generador nunca ve ninguna imagen real, ¡pero poco a poco aprende a producir imágenes falsas convincentes! Todo lo que obtiene son los gradientes que fluyen hacia atrás a través del discriminador. Afortunadamente, cuanto mejor se ponga el discriminador, más información sobre las imágenes reales está contenida en estos gradientes de segunda mano, por lo que el generador puede hacer un progreso significativo.

#### --------------------------------------------------------------------------------------------------------------

Sigamos adelante y construyamos un simple GAN para Fashion MNIST.

Primero, necesitamos construir el generador y el discriminador. El generador es similar al decodificador de un codificador automático y el discriminador es un clasificador binario normal: toma una imagen como entrada y termina con una capa densa que contiene una sola unidad y utiliza la función de activación sigmoidea. Para la segunda fase de cada iteración de entrenamiento, también necesitamos el modelo GAN completo que contiene el generador seguido del discriminador:

In [ ]:
codings_size = 30

Dense = tf.keras.layers.Dense
generator = tf.keras.Sequential([
    Dense(100, activation="relu", kernel_initializer="he_normal"),
    Dense(150, activation="relu", kernel_initializer="he_normal"),
    Dense(28 * 28, activation="sigmoid"),
    tf.keras.layers.Reshape([28, 28])
])
discriminator = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    Dense(150, activation="relu", kernel_initializer="he_normal"),
    Dense(100, activation="relu", kernel_initializer="he_normal"),
    Dense(1, activation="sigmoid")
])
gan = tf.keras.Sequential([generator, discriminator])

A continuación, necesitamos compilar estos modelos. Como el discriminador es un clasificador binario, naturalmente podemos utilizar la pérdida de entropía cruzada binaria. El modelo gan también es un clasificador binario, por lo que también puede utilizar la pérdida binaria de entropía cruzada. Sin embargo, el generador solo se entrenará a través del modelo `gan`, por lo que no es necesario compilarlo en absoluto. Es importante destacar que el discriminador no debe entrenarse durante la segunda fase, por lo que lo hacemos no entrenable antes de compilar el modelo gan:

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

#### NOTA

Keras solo tiene en cuenta el atributo entrenable al compilar un modelo, por lo que después de ejecutar este código, el `discriminator` se puede entrenar si llamamos a su método `fit()` o a su método `train_on_batch()` (que usaremos), mientras que no se puede entrenar cuando llamamos a estos métodos en el modelo `gan`.

#### -----------------------------------------------------------------------------------------------------------

Dado que el ciclo de entrenamiento es inusual, no podemos utilizar el método `fit()` normal. En su lugar, escribiremos un ciclo de entrenamiento personalizado. Para esto, primero necesitamos crear un conjunto de datos para recorrer las imágenes:

In [ ]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(buffer_size=1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

Ahora estamos listos para escribir el bucle de entrenamiento. Vamos a envolverlo en una función `train_gan()`:

In [ ]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        for X_batch in dataset:
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            gan.train_on_batch(noise, y2)

train_gan(gan, dataset, batch_size, codings_size, n_epochs=50)

Como se discutió anteriormente, puede ver las dos fases en cada iteración:

- En la fase uno, alimentamos ruido gaussiano al generador para producir imágenes falsas y completamos este lote concatenando un número igual de imágenes reales. Los objetivos `y1` se establecen en 0 para imágenes falsas y en 1 para imágenes reales. Luego entrenamos al discriminador en este lote. Recuerde que el discriminador es entrenable en esta fase, pero no estamos tocando el generador.

* En la fase dos, alimentamos a la GAN con algo de ruido gaussiano. Su generador comenzará produciendo imágenes falsas, luego el discriminador intentará adivinar si estas imágenes son falsas o reales. En esta fase, estamos intentando mejorar el generador, lo que significa que queremos que el discriminador falle: es por eso que los objetivos `y2` están todos configurados en 1, aunque las imágenes son falsas. En esta fase, el discriminador no se puede entrenar, por lo que la única parte del modelo `gan` que mejorará es el generador.

¡Eso es todo! Después del entrenamiento, puedes probar al azar algunas codificaciones de una distribución gaussiana y alimentarlas al generador para producir nuevas imágenes:

In [ ]:
codings = tf.random.normal(shape=[batch_size, codings_size])
generated_images = generator.predict(codings)

Si muestra las imágenes generadas (consulte la Figura 17-15), verá que al final de la primera época, ya comienzan a parecerse a imágenes de MNIST de moda (muy ruidosas).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1715.png)

(_Figura 17-15. Imágenes generadas por el GAN después de una época de entrenamiento_)

Desafortunadamente, las imágenes nunca son mucho mejores que eso, e incluso puedes encontrar épocas en las que el GAN parece estar olvidando lo que aprendió. ¿Por qué es eso? Bueno, resulta que entrenar a un GAN puede ser un desafío. Veamos por qué.

 
## Las dificultades de entrenar a los GAN


Durante el entrenamiento, el generador y el discriminador constantemente tratan de ser más astutos el uno del otro, en un juego de suma cero. A medida que avanza el entrenamiento, el juego puede terminar en un estado que los teóricos del juego llaman equilibrio de Nash, llamado así por el matemático John Nash: aquí es cuando ningún jugador estaría mejor cambiando su propia estrategia, suponiendo que los otros jugadores no cambien la suya. Por ejemplo, se alcanza un equilibrio de Nash cuando todo el mundo conduce por el lado izquierdo de la carretera: ningún conductor sería mejor ser el único que cambia de bando. Por supuesto, hay un segundo equilibrio posible de Nash: cuando todos conducen por el lado derecho de la carretera. Los diferentes estados iniciales y dinámicas pueden conducir a un equilibrio u otro. En este ejemplo, hay una sola estrategia óptima una vez que se alcanza un equilibrio (es decir, conducir del mismo lado que todos los demás), pero un equilibrio de Nash puede implicar múltiples estrategias competitivas (por ejemplo, un depredador persigue a su presa, la presa intenta escapar, y ninguno de los dos sería mejor cambiar su estrategia).

Entonces, ¿cómo se aplica esto a los GAN? Bueno, los autores del documento GAN demostraron que un GAN solo puede alcanzar un solo equilibrio de Nash: ahí es cuando el generador produce imágenes perfectamente realistas, y el discriminador se ve obligado a adivinar (50% real, 50% falso). Este hecho es muy alentador: parece que solo necesitas entrenar al GAN el tiempo suficiente, y eventualmente alcanzará este equilibrio, lo que te dará un generador perfecto. Desafortunadamente, no es tan simple: nada garantiza que se alcance el equilibrio.

La mayor dificultad se llama colapso del modo: es cuando las salidas del generador se vuelven gradualmente menos diversas. ¿Cómo puede suceder esto? Supongamos que el generador es mejor en la producción de zapatos convincentes que cualquier otra clase. Engañará un poco más al discriminador con los zapatos, y esto lo animará a producir aún más imágenes de los zapatos. Poco a poco, se olvidará de cómo producir cualquier otra cosa. Mientras tanto, las únicas imágenes falsas que el discriminador verá serán los zapatos, por lo que también olvidará cómo discriminar las imágenes falsas de otras clases. Eventualmente, cuando el discriminador se las arregle para discriminar los zapatos falsos de los reales, el generador se verá obligado a pasar a otra clase. Entonces puede llegar a ser bueno en las camisas, olvidándose de los zapatos, y el discriminador lo seguirá. El GAN puede circular gradualmente a través de algunas clases, sin llegar a ser realmente muy bueno en ninguna de ellas.

Además, debido a que el generador y el discriminador se presionan constantemente entre sí, sus parámetros pueden terminar oscilando y volverse inestables. El entrenamiento puede comenzar correctamente, luego divergir de repente sin razón aparente, debido a estas inestabilidades. Y dado que muchos factores afectan a estas dinámicas complejas, los GAN son muy sensibles a los hiperparámetros: es posible que tengas que dedicar mucho esfuerzo a ajustarlos. De hecho, es por eso que usé RMSProp en lugar de Nadam al compilar los modelos: al usar Nadam, me encontré con un grave colapso del modo.

Estos problemas han mantenido a los investigadores muy ocupados desde 2014: se han publicado muchos artículos sobre este tema, algunos proponiendo nuevas funciones de costo⁠10 (aunque un documento de 2018⁠11 de los investigadores de Google cuestiona su eficiencia) o técnicas para estabilizar la capacitación o para evitar el problema del colapso del modo. Por ejemplo, una técnica popular llamada repetición de la experiencia consiste en almacenar las imágenes producidas por el generador en cada iteración en un búfer de reproducción (dejar caer gradualmente las imágenes generadas más antiguas) y entrenar al discriminador utilizando imágenes reales más imágenes falsas extraídas de este búfer (en lugar de solo imágenes falsas producidas por el generador actual). Esto reduce las posibilidades de que el discriminador sobreajuste las salidas del último generador. Otra técnica común se llama discriminación en mini lotes: mide cómo son las imágenes similares en todo el lote y proporciona esta estadística al discriminador, por lo que puede rechazar fácilmente todo un lote de imágenes falsas que carecen de diversidad. Esto anima al generador a producir una mayor variedad de imágenes, lo que reduce la posibilidad de colapso del modo. Otros artículos simplemente proponen arquitecturas específicas que funcionan bien.

En resumen, este sigue siendo un campo de investigación muy activo, y la dinámica de los GAN todavía no se entiende perfectamente. Pero la buena noticia es que se ha hecho un gran progreso, ¡y algunos de los resultados son realmente asombrosos! Así que echemos un vistazo a algunas de las arquitecturas más exitosas, comenzando con los GAN convolucionales profundos, que eran el estado del arte hace solo unos años. Luego analizaremos dos arquitecturas más recientes (y más complejas).


## GAN convolucionales profundos


Los autores del artículo original de GAN experimentaron con capas convolucionales, pero solo intentaron generar imágenes pequeñas. Poco después, muchos investigadores intentaron construir GAN basadas en redes convolucionales más profundas para imágenes más grandes. Esto resultó ser complicado, ya que el entrenamiento era muy inestable, pero Alec Radford et al. finalmente tuvieron éxito a finales de 2015, después de experimentar con muchas arquitecturas e hiperparámetros diferentes. Llamaron a su arquitectura GAN convolucionales profundos (DCGAN). Estas son las principales directrices que propusieron para construir GAN convolucionales estables:

- Reemplace cualquier capa de agrupación con convoluciones estrilladas (en el discriminador) y convoluciones transpuestas (en el generador).

* Utilice la normalización por lotes tanto en el generador como en el discriminador, excepto en la capa de salida del generador y en la capa de entrada del discriminador.

- Elimina las capas ocultas totalmente conectadas para arquitecturas más profundas.

* Utilice la activación de ReLU en el generador para todas las capas, excepto la capa de salida, que debería usar tanh.

- Utilice la activación de ReLU con fugas en el discriminador para todas las capas.

Estas pautas funcionarán en muchos casos, pero no siempre, por lo que es posible que aún tenga que experimentar con diferentes hiperparámetros. De hecho, solo cambiar la semilla al azar y volver a entrenar exactamente el mismo modelo a veces funcionará. Aquí hay un pequeño DCGAN que funciona razonablemente bien con Fashion MNIST:

In [ ]:
codings_size = 100

generator = tf.keras.Sequential([
    tf.keras.layers.Dense(7 * 7 * 128),
    tf.keras.layers.Reshape([7, 7, 128]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2,
                                    padding="same", activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2,
                                    padding="same", activation="tanh"),
])
discriminator = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="same",
                           activation=tf.keras.layers.LeakyReLU(0.2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="same",
                           activation=tf.keras.layers.LeakyReLU(0.2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
gan = tf.keras.Sequential([generator, discriminator])

El generador toma codificaciones de tamaño 100, las proyecta a 6.272 dimensiones (7 * 7 * 128) y remodela el resultado para obtener un tensor de 7 × 7 × 128. Este tensor se normaliza por lotes y se alimenta a una capa convolucional transpuesta con un paso de 2, que lo muestrea de 7 × 7 a 14 × 14 y reduce su profundidad de 128 a 64. El resultado es un lote normalizado de nuevo y alimentado a otra capa convolucional transpuesta con un paso de 2, que lo muestrea de 14 × 14 a 28 × 28 y reduce la profundidad de 64 a 1. Esta capa utiliza la función de activación de tanh, por lo que las salidas oscilarán entre -1 y 1. Por esta razón, antes de entrenar al GAN, necesitamos reescalar el conjunto de entrenamiento a ese mismo rango. También necesitamos remodelarlo para añadir la dimensión del canal:

In [ ]:
X_train_dcgan = X_train.reshape(-1, 28, 28, 1) * 2. - 1. # reshape and rescale

El discriminador se parece mucho a una CNN normal para clasificación binaria, excepto que en lugar de usar capas de agrupación máxima para reducir la resolución de la imagen, usamos convoluciones con zancadas (`strides=2`). Tenga en cuenta que utilizamos la función de activación ReLU con fugas. En general, respetamos las pautas de DCGAN, excepto que reemplazamos las capas `BatchNormalization` en el discriminador con capas `Dropout`; de lo contrario, en este caso el entrenamiento fue inestable. Siéntase libre de modificar esta arquitectura: verá cuán sensible es a los hiperparámetros, especialmente las tasas de aprendizaje relativas de las dos redes.

Por último, para construir el conjunto de datos y luego compilar y entrenar este modelo, podemos usar el mismo código que antes. Después de 50 épocas de entrenamiento, el generador produce imágenes como las que se muestran en la Figura 17-16. Todavía no es perfecto, pero muchas de estas imágenes son bastante convincentes.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1716.png)

(_Figura 17-16. Imágenes generadas por el DCGAN después de 50 épocas de entrenamiento_)

Si amplías esta arquitectura y la entrenas en un gran conjunto de datos de caras, puedes obtener imágenes bastante realistas. De hecho, los DCGAN pueden aprender representaciones latentes bastante significativas, como se puede ver en la Figura 17-17: se generaron muchas imágenes, y nueve de ellas fueron elegidas manualmente (arriba a la izquierda), incluyendo tres que representan a hombres con gafas, tres hombres sin gafas y tres mujeres sin gafas. Para cada una de estas categorías, se promediaron las codificaciones que se utilizaron para generar las imágenes, y se generó una imagen basada en las codificaciones medias resultantes (abajo a la izquierda). En resumen, cada una de las tres imágenes de la parte inferior izquierda representa la media de las tres imágenes ubicadas encima de ella. Pero esta no es una media simple calculada a nivel de píxel (esto daría lugar a tres caras superpuestas), es una media calculada en el espacio latente, por lo que las imágenes todavía se ven como caras normales. Sorprendentemente, si calculas a los hombres con gafas, menos a los hombres sin gafas, más las mujeres sin gafas, donde cada término corresponde a una de las codificaciones medias, y generas la imagen que corresponde a esta codificación, obtienes la imagen en el centro de la cuadrícula 3 × 3 de caras a la derecha: ¡una mujer con gafas! Las otras ocho imágenes a su alrededor se generaron en base al mismo vector más un poco de ruido, para ilustrar las capacidades de interpolación semántica de los DCGAN. ¡Ser capaz de hacer aritmética en las caras se siente como ciencia ficción!

Sin embargo, los DCGAN no son perfectos. Por ejemplo, cuando intentas generar imágenes muy grandes usando DCGAN, a menudo terminas con características localmente convincentes pero inconsistencias generales, como camisas con una manga mucho más larga que la otra, diferentes pendientes o ojos que miran en direcciones opuestas. ¿Cómo puedes arreglar esto?

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1717.png)

(_Figura 17-17. Aritmética vectorial para conceptos visuales (parte de la figura 7 del documento DCGAN)⁠_)

#### PROPINA

Si agrega la clase de cada imagen como una entrada adicional tanto al generador como al discriminador, ambos aprenderán cómo se ve cada clase y, por lo tanto, podrá controlar la clase de cada imagen producida por el generador. Esto se llama GAN condicional (CGAN).

#### ----------------------------------------------------------------------------------------------------------

## Crecimiento progresivo de GAN

En un artículo de 2018,⁠ investigadores de Nvidia Tero Kerras et al. propusieron una técnica importante: sugirieron generar imágenes pequeñas al comienzo del entrenamiento, luego agregar gradualmente capas convolucionales tanto al generador como al discriminador para producir imágenes cada vez más grandes (4 × 4, 8 × 8, 16 × 16, ... , 512 × 512, 1,024 × 1,024). Este enfoque se asemeja al entrenamiento codicioso en capas de autocodificadores apilados. Las capas adicionales se añaden al final del generador y al principio del discriminador, y las capas previamente entrenadas siguen siendo entrenables.

Por ejemplo, al aumentar las salidas del generador de 4 × 4 a 8 × 8 (consulte la Figura 17-18), se agrega una capa de muestreo superior (usando el filtrado del vecino más cercano) a la capa convolucional existente (“Conv 1”) para producir 8 × 8 mapas de características. Estos se envían a la nueva capa convolucional (“Conv 2”), que a su vez se alimenta a una nueva capa convolucional de salida. Para evitar romper los pesos entrenados de Conv 1, gradualmente desvanecemos las dos nuevas capas convolucionales (representadas con líneas discontinuas en la Figura 17-18) y desvanecemos la capa de salida original. Las salidas finales son una suma ponderada de las nuevas salidas (con peso α) y las salidas originales (con peso 1 – α), aumentando lentamente α de 0 a 1. Se utiliza una técnica de aparición/desaparición similar cuando Se agrega una nueva capa convolucional al discriminador (seguida de una capa de agrupación promedio para la reducción de resolución). Tenga en cuenta que todas las capas convolucionales utilizan el "mismo" relleno y pasos de 1, por lo que conservan la altura y el ancho de sus entradas. Esto incluye la capa convolucional original, por lo que ahora produce salidas de 8 × 8 (ya que sus entradas ahora son 8 × 8). Por último, las capas de salida utilizan el tamaño de kernel 1. Simplemente proyectan sus entradas hasta el número deseado de canales de color (normalmente 3).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1718.png)

(_Figura 17-18. Un GAN que crece progresivamente: un generador GAN genera 4 × 4 imágenes en color (izquierda); lo ampliamos para generar 8 × 8 imágenes (derecha)_)


El documento también introdujo varias otras técnicas destinadas a aumentar la diversidad de los resultados (para evitar el colapso del modo) y hacer que el entrenamiento sea más estable:

**Capa de desviación estándar de mini lote**

    Añadido cerca del final del discriminador. Para cada posición en las entradas, calcula la desviación estándar en todos los canales y todas las instancias del lote (S = tf.math.reduce_std(inputs, axis=[0, -1])). Estas desviaciones estándar se promedian en todos los puntos para obtener un solo valor (v = tf.reduce_​mean(S)). Finalmente, se agrega un mapa de características adicional a cada instancia del lote y se rellena con el valor calculado (tf.concat([inputs, tf.fill([batch_size, height, width, 1], v)], axis=-1)). ¿Cómo ayuda esto? Bueno, si el generador produce imágenes con poca variedad, entonces habrá una pequeña desviación estándar a través de los mapas de características en el discriminador. Gracias a esta capa, el discriminador tendrá fácil acceso a esta estadística, por lo que es menos probable que sea engañado por un generador que produce muy poca diversidad. Esto alentará al generador a producir salidas más diversas, reduciendo el riesgo de colapso del modo.

**Tasa de aprendizaje igualada**

    Inicializa todos los pesos usando una distribución gaussiana con media 0 y desviación estándar 1 en lugar de usar la inicialización He. Sin embargo, los pesos se reducen en tiempo de ejecución (es decir, cada vez que se ejecuta la capa) por el mismo factor que en la inicialización: se dividen por sqrt(2/N entradas), donde ninputs es el número de entradas a la capa. El documento demostró que esta técnica mejoró significativamente el rendimiento de la GAN al usar RMSProp, Adam u otros optimizadores de gradiente adaptativo. De hecho, estos optimizadores normalizan las actualizaciones de gradiente por su desviación estándar estimada (ver Capítulo 11), por lo que los parámetros que tienen un rango dinámico mayor⁠ tardarán más en entrenarse, mientras que los parámetros con un rango dinámico pequeño pueden actualizarse demasiado rápido, lo que lleva a inestabilidades. Al reescalar los pesos como parte del modelo en sí en lugar de simplemente volver a escalarlos al inicializar, este enfoque garantiza que el rango dinámico sea el mismo para todos los parámetros a lo largo del entrenamiento, para que todos aprendan a la misma velocidad. Esto acelera y estabiliza el entrenamiento.

**Capa de normalización por píxeles**

    Agregado después de cada capa convolucional en el generador. Normaliza cada activación en función de todas las activaciones en la misma imagen y en la misma ubicación, pero en todos los canales (dividiendo por la raíz cuadrada de la activación cuadrática media). En el código de TensorFlow, esto es inputs / tf.sqrt(tf.reduce_mean(tf.square(X), axis=-1, keepdims=True) + 1e-8) (el término de suavizado 1e-8 es necesario para evitar la división por cero). Esta técnica evita explosiones en las activaciones por excesiva competencia entre el generador y el discriminador.
    

La combinación de todas estas técnicas permitió a los autores generar imágenes de alta definición extremadamente convincentes de caras. Pero, ¿qué llamamos exactamente "convincer"? La evaluación es uno de los grandes desafíos cuando se trabaja con GAN: aunque es posible evaluar automáticamente la diversidad de las imágenes generadas, juzgar su calidad es una tarea mucho más complicada y subjetiva. Una técnica es usar tasadores humanos, pero esto es costoso y requiere mucho tiempo. Por lo tanto, los autores propusieron medir la similitud entre la estructura de la imagen local de las imágenes generadas y las imágenes de entrenamiento, teniendo en cuenta cada escala. Esta idea los llevó a otra innovación innovadora: StyleGANs.

## StyleGANs

El estado del arte en la generación de imágenes de alta resolución fue avanzado una vez más por el mismo equipo de Nvidia en un documento de 2018⁠ que introdujo la popular arquitectura StyleGAN. Los autores utilizaron técnicas de transferencia de estilo en el generador para garantizar que las imágenes generadas tengan la misma estructura local que las imágenes de entrenamiento, en cada escala, mejorando en gran medida la calidad de las imágenes generadas. El discriminador y la función de pérdida no se modificaron, solo el generador. Un generador StyleGAN se compone de dos redes (ver Figura 17-19):

**Mapeo de la red**

    Un MLP de ocho capas que asigna las representaciones latentes z (es decir, las codificaciones) a un vector w. Luego, este vector se envía a través de múltiples transformaciones afines (es decir, capas densas sin funciones de activación, representadas por los cuadros "A" en la Figura 17-19), lo que produce múltiples vectores. Estos vectores controlan el estilo de la imagen generada en diferentes niveles, desde textura fina (por ejemplo, color de cabello) hasta características de alto nivel (por ejemplo, adulto o niño). En resumen, la red de mapeo asigna las codificaciones a múltiples vectores de estilo.

**Red de síntesis**

    Responsable de generar las imágenes. Tiene una entrada de aprendizaje constante (para que quede claro, esta entrada será constante después del entrenamiento, pero durante el entrenamiento sigue siendo ajustada por la repropagación). Procesa esta entrada a través de múltiples capas de convolucionación y muestreo ascendente, como antes, pero hay dos giros. En primer lugar, se añade algo de ruido a la entrada y a todas las salidas de las capas convolucionales (antes de la función de activación). En segundo lugar, cada capa de ruido va seguida de una capa de normalización de instancia adaptativa (AdaIN): estandariza cada mapa de características de forma independiente (restando la media del mapa de características y dividiéndola por su desviación estándar), luego utiliza el vector de estilo para determinar la escala y el desplazamiento de cada mapa de características (el vector de estilo contiene una escala y un término de sesgo para cada mapa de características).


![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1719.png)

(_Figura 17-19. Arquitectura del generador de StyleGAN (parte de la Figura 1 del documento StyleGAN)⁠_)

La idea de añadir ruido independientemente de las codificaciones es muy importante. Algunas partes de una imagen son bastante aleatorias, como la posición exacta de cada peca o cabello. En los GAN anteriores, esta aleatoriedad tenía que venir de las codificaciones o ser algún ruido pseudoaleatorio producido por el propio generador. Si venía de las codificaciones, significaba que el generador tenía que dedicar una parte significativa del poder de representación de las codificaciones a almacenar ruido, lo cual es bastante derrochador. Además, el ruido tenía que ser capaz de fluir a través de la red y llegar a las capas finales del generador: esto parece una restricción innecesaria que probablemente ralenticó el entrenamiento. Y, por último, algunos artefactos visuales pueden aparecer porque el mismo ruido se utilizó en diferentes niveles. Si, en cambio, el generador intentara producir su propio ruido pseudoaleatorio, este ruido podría no parecer muy convincente, lo que llevaría a más artefactos visuales. Además, parte de los pesos del generador se dedicarían a generar ruido pseudoaleatorio, que de nuevo parece derrochador. Al agregar entradas de ruido adicionales, se evitan todos estos problemas; el GAN puede usar el ruido proporcionado para agregar la cantidad correcta de estocasticidad a cada parte de la imagen.

El ruido añadido es diferente para cada nivel. Cada entrada de ruido consiste en un solo mapa de características lleno de ruido gaussiano, que se transmite a todos los mapas de características (del nivel dado) y se escala utilizando factores de escalado por característica aprendidos (esto está representado por los cuadros "B" en la Figura 17-19) antes de que se agregue.

Por último, StyleGAN utiliza una técnica llamada regularización de mezcla (o mezcla de estilo), en la que un porcentaje de las imágenes generadas se producen utilizando dos codificaciones diferentes. Específicamente, las codificaciones c1 y c2 se envían a través de la red de mapeo, dando dos vectores de estilo w1 y w2. Luego, la red de síntesis genera una imagen basada en los estilos w1 para los primeros niveles y los estilos w2 para los niveles restantes. El nivel de corte se elige al azar. Esto evita que la red asuma que los estilos en los niveles adyacentes están correlacionados, lo que a su vez fomenta la localidad en el GAN, lo que significa que cada vector de estilo solo afecta a un número limitado de rasgos en la imagen generada.

Hay una variedad tan amplia de GAN por ahí que se necesitaría un libro entero para cubrirlos a todos. Esperemos que esta introducción te haya dado las ideas principales y, lo que es más importante, el deseo de aprender más. Adelante, implementa tu propio GAN, y no te desanimes si tiene problemas para aprender al principio: desafortunadamente, esto es normal, y requerirá un poco de paciencia para que funcione, pero el resultado vale la pena. Si tienes problemas con un detalle de implementación, hay muchas implementaciones de Keras o TensorFlow que puedes ver. De hecho, si todo lo que quieres es obtener resultados increíbles rápidamente, entonces puedes usar un modelo preentrenado (por ejemplo, hay modelos StyleGAN preentrenados disponibles para Keras).

Ahora que hemos examinado los autocodificadores y los GAN, echemos un vistazo a un último tipo de arquitectura: los modelos de difusión.


# Modelos de difusión

Las ideas detrás de los modelos de difusión han existido durante muchos años, pero se formalizaron por primera vez en su forma moderna en un artículo de 2015⁠ de Jascha Sohl-Dickstein et al. de la Universidad de Stanford y la UC Berkeley. Los autores aplicaron herramientas de la termodinámica para modelar un proceso de difusión, similar a una gota de leche que se difunde en una taza de té. La idea central es entrenar a un modelo para aprender el proceso inverso: comenzar desde el estado completamente mezclado y "desmezclar" gradualmente la leche del té. Usando esta idea, obtuvieron resultados prometedores en la generación de imágenes, pero como los GAN produjeron imágenes más convincentes en ese entonces, los modelos de difusión no obtuvieron tanta atención.

Luego, en 2020, Jonathan Ho et al., también de UC Berkeley, lograron construir un modelo de difusión capaz de generar imágenes altamente realistas, que llamaron un modelo probabilístico de difusión de desnadización (DDPM). Unos meses más tarde, un artículo de 2021⁠21 de los investigadores de OpenAI Alex Nichol y Prafulla Dhariwal analizó la arquitectura de DDPM y propuso varias mejoras que permitieron a los DDPM finalmente vencer a los GAN: los DDPM no solo son mucho más fáciles de entrenar que los GAN, sino que las imágenes generadas son más diversas y de aún mayor calidad. La principal desventaja de los DDPM, como verás, es que tardan mucho tiempo en generar imágenes, en comparación con los GAN o VAE.

Entonces, ¿cómo funciona exactamente un DDPM? Bueno, supongamos que comienzas con una imagen de un gato (como la que verás en la Figura 17-20), anotada x0, y en cada paso t agregas un poco de ruido gaussiano a la imagen, con una media de 0 y una varianza βt. Este ruido es independiente para cada píxel: lo llamamos isotrópico. Primero obtienes la imagen x1, thenx2, y así sucesivamente, hasta que el gato esté completamente oculto por el ruido, imposible de ver. El último paso de tiempo se observa T. En el documento original de DDPM, los autores usaron T = 1.000, y programaron la variación βt de tal manera que la señal de gato se desvanece linealmente entre los pasos de tiempo 0 y T. En el documento mejorado de DDPM, T se aumentó a 4.000, y el programa de variación se ajustó para cambiar más lentamente al principio y al final. En resumen, estamos ahogando gradualmente al gato en el ruido: esto se llama el proceso de avance.

A medida que añadimos más y más ruido gaussiano en el proceso de avance, la distribución de los valores de los píxeles se vuelve cada vez más gaussiana. Un detalle importante que dejé fuera es que los valores de los píxeles se reescalan ligeramente en cada paso, por un factor de sqrt(1-βT). Esto asegura que la media de los valores de los píxeles se acerque gradualmente a 0, ya que el factor de escala es un poco menor que 1 (imagine multiplicar repetidamente un número por 0,99). También asegura que la variación convergerá gradualmente a 1. Esto se debe a que la desviación estándar de los valores de los píxeles también se escala por sqrt(1-βT), por lo que la varianza se escala en 1 - βt (es decir, el cuadrado del factor de escala). Pero la varianza no puede reducirse a 0, ya que estamos agregando ruido gaussiano con la varianza βt en cada paso. Y dado que las variaciones se suman cuando se suman las distribuciones gaussianas, se puede ver que la varianza solo puede converger a 1 - βt + βt = 1.

El proceso de difusión hacia adelante se resume en la ecuación 17-5. Esta ecuación no te enseñará nada nuevo sobre el proceso de avance, pero es útil para entender este tipo de notación matemática, ya que se usa a menudo en los documentos de aprendizaje automático. Esta ecuación define la distribución de probabilidad q de xt dada xt-1 como una distribución gaussiana con la media xt-1 veces el factor de escala, y con una matriz de covarianza igual a βtI. Esta es la matriz de identidad que se multiplica por βt, lo que significa que el ruido es isotrópico con la varianza βt.


### Ecuación 17-5. Distribución de probabilidad q del proceso de difusión hacia adelante

<a href="https://imgbb.com/"><img src="https://i.ibb.co/j6JmZhG/Captura-de-pantalla-2024-03-30-a-las-6-15-18.png" alt="Captura-de-pantalla-2024-03-30-a-las-6-15-18" border="0"></a>

Curiosamente, hay un acceso directo para el proceso de avance: es posible muestrear una imagen xt dada x0 sin tener que calcular primero x1, x2, ... , xt-1. De hecho, dado que la suma de múltiples distribuciones gaussianas también es una distribución gaussiana, todo el ruido se puede agregar en una sola toma usando la ecuación 17-6. Esta es la ecuación que usaremos, ya que es mucho más rápida.


### Ecuación 17-6. Acceso directo para el proceso de difusión hacia adelante

<a href="https://imgbb.com/"><img src="https://i.ibb.co/YL7qfvK/Captura-de-pantalla-2024-03-30-a-las-6-16-13.png" alt="Captura-de-pantalla-2024-03-30-a-las-6-16-13" border="0"></a>

Nuestro objetivo, por supuesto, no es ahogar a los gatos en el ruido. Por el contrario, ¡queremos crear muchos gatos nuevos! Podemos hacerlo entrenando a un modelo que pueda realizar el proceso inverso: pasar de xt a xt-1. Luego podemos usarlo para eliminar un poco de ruido de una imagen, y repetir la operación muchas veces hasta que todo el ruido haya desaparecido. Si entrenamos el modelo en un conjunto de datos que contiene muchas imágenes de gatos, entonces podemos darle una imagen completamente llena de ruido gaussiano, y el modelo hará que aparezca gradualmente un gato nuevo (ver Figura 17-20).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1720.png)

(_Figura 17-20. El proceso hacia adelante q y el proceso inverso p_)

Vale, ¡así que empecemos a programar! Lo primero que tenemos que hacer es codificar el proceso de avance. Para esto, primero tendremos que implementar el calendario de variación. ¿Cómo podemos controlar la rapidez con la que desaparece el gato? Inicialmente, el 100 % de la variación proviene de la imagen original del gato. Luego, en cada paso de tiempo t, la varianza se multiplica por 1 -βt, como se explicó anteriormente, y se añade ruido. Por lo tanto, la parte de la varianza que proviene de la distribución inicial se reduce en un factor de 1 - βt en cada paso. Si definimos αt = 1 - βt, entonces después de t pasos de tiempo, la señal de gato se habrá multiplicado por un factor de α̅t = α1×α2×... ×αt =α¯T=∏yo=1TαT. Es este factor de "señal de gato" α̅t el que queremos programar para que se reduzca de 1 a 0 gradualmente entre los pasos de tiempo 0 y T. En el documento mejorado de DDPM, los autores programan α̅t de acuerdo con la ecuación 17-7. Este horario está representado en la Figura 17-21.


### Ecuación 17-7. Ecuaciones del programa de variación para el proceso de difusión hacia adelante

<a href="https://imgbb.com/"><img src="https://i.ibb.co/frBvFYb/Captura-de-pantalla-2024-03-30-a-las-6-20-01.png" alt="Captura-de-pantalla-2024-03-30-a-las-6-20-01" border="0"></a>

En estas ecuaciones:

- s es un valor minúsculo que evita que βt sea demasiado pequeño cerca de t = 0. En el artículo, los autores usaron s = 0,008.

* βt se recorta para que no sea mayor que 0,999, para evitar inestabilidades cerca de t = T.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1721.png)

(_Figura 17-21. Programa de variación de ruido βt, y la varianza de señal restante α̅t_)

Vamos a crear una pequeña función para calcular αt, βt y α̅t, y llamarla con T = 4.000:

In [ ]:
def variance_schedule(T, s=0.008, max_beta=0.999):
    t = np.arange(T + 1)
    f = np.cos((t / T + s) / (1 + s) * np.pi / 2) ** 2
    alpha = np.clip(f[1:] / f[:-1], 1 - max_beta, 1)
    alpha = np.append(1, alpha).astype(np.float32)  # add α₀ = 1
    beta = 1 - alpha
    alpha_cumprod = np.cumprod(alpha)
    return alpha, alpha_cumprod, beta  # αₜ , α̅ₜ , βₜ for t = 0 to T

T = 4000
alpha, alpha_cumprod, beta = variance_schedule(T)

Para entrenar nuestro modelo para revertir el proceso de difusión, necesitaremos imágenes ruidosas de diferentes pasos de tiempo del proceso directo. Para esto, creemos una función `prepare_batch()` que tomará un lote de imágenes limpias del conjunto de datos y las preparará:

In [ ]:
def prepare_batch(X):
    X = tf.cast(X[..., tf.newaxis], tf.float32) * 2 - 1  # scale from –1 to +1
    X_shape = tf.shape(X)
    t = tf.random.uniform([X_shape[0]], minval=1, maxval=T + 1, dtype=tf.int32)
    alpha_cm = tf.gather(alpha_cumprod, t)
    alpha_cm = tf.reshape(alpha_cm, [X_shape[0]] + [1] * (len(X_shape) - 1))
    noise = tf.random.normal(X_shape)
    return {
        "X_noisy": alpha_cm ** 0.5 * X + (1 - alpha_cm) ** 0.5 * noise,
        "time": t,
    }, noise

Vamos a revisar este código:

- Para simplificar, utilizaremos Fashion MNIST, por lo que la función primero debe agregar un eje de canal. También ayudará a escalar los valores de los píxeles de -1 a 1, por lo que está más cerca de la distribución gaussiana final con la media 0 y la varianza 1.

* A continuación, la función crea t, un vector que contiene un paso de tiempo aleatorio para cada imagen en el lote, entre 1 y T.

- Luego usa `tf.gather()` para obtener el valor de `alpha_cumprod` para cada uno de los pasos de tiempo en el vector `t`. Esto nos da el vector `alpha_cm`, que contiene un valor de α̅t para cada imagen.

* La siguiente línea cambia la forma de `alpha_cm` de [tamaño de lote] a [tamaño de lote, 1, 1, 1]. Esto es necesario para garantizar que `alpha_cm` se pueda transmitir con el lote X.

- Luego generamos algo de ruido gaussiano con media 0 y varianza 1.

* Por último, utilizamos la ecuación 17-6 para aplicar el proceso de difusión a las imágenes. Tenga en cuenta que `x ** 0.5` es igual a la raíz cuadrada de `x`. La función devuelve una tupla que contiene las entradas y los objetivos. Las entradas se representan como un dictado de Python que contiene las imágenes ruidosas y los pasos de tiempo utilizados para generarlas. Los objetivos son el ruido gaussiano utilizado para generar cada imagen.


#### NOTA

Con esta configuración, el modelo predecirá el ruido que se debe restar de la imagen de entrada para obtener la imagen original. ¿Por qué no predecir la imagen original directamente? Bueno, los autores lo intentaron: simplemente no funciona tan bien.

#### -----------------------------------------------------------------------------------------------------------


A continuación, crearemos un conjunto de datos de entrenamiento y un conjunto de validación que aplicará la función `prepare_batch()` a cada lote. Como antes, `X_train` y `X_valid` contienen las imágenes Fashion MNIST con valores de píxeles que van de 0 a 1:

In [ ]:
def prepare_dataset(X, batch_size=32, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(X)
    if shuffle:
        ds = ds.shuffle(buffer_size=10_000)
    return ds.batch(batch_size).map(prepare_batch).prefetch(1)

train_set = prepare_dataset(X_train, batch_size=32, shuffle=True)
valid_set = prepare_dataset(X_valid, batch_size=32)

Ahora estamos listos para construir el modelo de difusión en sí mismo. Puede ser cualquier modelo que desee, siempre y cuando tome las imágenes ruidosas y los pasos de tiempo como entradas, y prediga el ruido para restar de las imágenes de entrada:

In [ ]:
def build_diffusion_model():
    X_noisy = tf.keras.layers.Input(shape=[28, 28, 1], name="X_noisy")
    time_input = tf.keras.layers.Input(shape=[], dtype=tf.int32, name="time")
    [...]  # build the model based on the noisy images and the time steps
    outputs = [...]  # predict the noise (same shape as the input images)
    return tf.keras.Model(inputs=[X_noisy, time_input], outputs=[outputs])

Los autores de DDPM utilizaron una arquitectura U-Net modificada,⁠ que tiene muchas similitudes con la arquitectura FCN que analizamos en el Capítulo 14 para la segmentación semántica: es una red neuronal convolucional que reduce gradualmente la muestra de las imágenes de entrada y luego las vuelve a aumentar gradualmente, con saltos. conexiones que se cruzan desde cada nivel de la parte de muestreo descendente al nivel correspondiente en la parte de muestreo de aumento. Para tener en cuenta los pasos de tiempo, los codificaron utilizando la misma técnica que las codificaciones posicionales en la arquitectura del transformador (ver Capítulo 16). En todos los niveles de la arquitectura U-Net, pasaron estas codificaciones de tiempo a través de capas densas y las alimentaron a U-Net. Por último, también utilizaron capas de atención de múltiples cabezas en varios niveles. Consulte el cuaderno de este capítulo para ver una implementación básica, o https://homl.info/ddpmcode para ver la implementación oficial: está basado en TF 1.x, que está en desuso, pero es bastante legible.

Ahora podemos entrenar al modelo normalmente. Los autores señalaron que el uso de la pérdida de MAE funcionó mejor que el MSE. También puedes usar la pérdida de Huber:

In [ ]:
model = build_diffusion_model()
model.compile(loss=tf.keras.losses.Huber(), optimizer="nadam")
history = model.fit(train_set, validation_data=valid_set, epochs=100)

Una vez que se entrena el modelo, puedes usarlo para generar nuevas imágenes. Desafortunadamente, no hay ningún acceso directo en el proceso de difusión inversa, por lo que tiene que muestrear xT al azar de una distribución gaussiana con media 0 y varianza 1, luego pasarlo al modelo para predecir el ruido; restarlo de la imagen usando la ecuación 17-8, y obtienes xT-1. Repite el proceso 3.999 veces más hasta que obtengas x0: si todo salió bien, ¡debería parecer una imagen normal de Fashion MNIST!

### Ecuación 17-8. Ir un paso atrás en el proceso de difusión

<a href="https://imgbb.com/"><img src="https://i.ibb.co/hsLnwbn/Captura-de-pantalla-2024-03-30-a-las-6-26-36.png" alt="Captura-de-pantalla-2024-03-30-a-las-6-26-36" border="0"></a><br /><a target='_blank' href='https://es.imgbb.com/'>avatar imagenes</a><br />

En esta ecuación, εθ(xt, t) representa el ruido predicho por el modelo dado la imagen de entrada xt y el paso de tiempo t. El θ representa los parámetros del modelo. Además, z es ruido gaussiano con media 0 y varianza 1. Esto hace que el proceso inverso sea estocástico: si lo ejecutas varias veces, obtendrás diferentes imágenes.

Escribamos una función que implemente este proceso inverso y llamémosla para generar algunas imágenes:

In [ ]:
def generate(model, batch_size=32):
    X = tf.random.normal([batch_size, 28, 28, 1])
    for t in range(T, 0, -1):
        noise = (tf.random.normal if t > 1 else tf.zeros)(tf.shape(X))
        X_noise = model({"X_noisy": X, "time": tf.constant([t] * batch_size)})
        X = (
            1 / alpha[t] ** 0.5
            * (X - beta[t] / (1 - alpha_cumprod[t]) ** 0.5 * X_noise)
            + (1 - alpha[t]) ** 0.5 * noise
        )
    return X

X_gen = generate(model)  # generated images

Esto puede tardar uno o dos minutos. Ese es el principal inconveniente de los modelos de difusión: la generación de imágenes es lenta, ya que el modelo necesita ser llamado muchas veces. Es posible hacer esto más rápido utilizando un valor T más pequeño, o utilizando la misma predicción del modelo durante varios pasos a la vez, pero las imágenes resultantes pueden no verse tan bien. Dicho esto, a pesar de esta limitación de velocidad, los modelos de difusión producen imágenes diversas y de alta calidad, como se puede ver en la Figura 17-22.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1722.png)

(_Figura 17-22. Imágenes generadas por el DDPM_)

Los modelos de difusión han hecho un tremendo progreso recientemente. En particular, un artículo publicado en diciembre de 2021 por Robin Rombach, Andreas Blattmann, et al.,⁠23 introdujo modelos de difusión latente, donde el proceso de difusión tiene lugar en el espacio latente, en lugar de en el espacio de píxeles. Para lograr esto, se utiliza un potente autoencoder para comprimir cada imagen de entrenamiento en un espacio latente mucho más pequeño, donde se lleva a cabo el proceso de difusión, luego se utiliza el autoencoder para descomprimir la representación latente final, generando la imagen de salida. Esto acelera considerablemente la generación de imágenes y reduce drásticamente el tiempo y el costo de entrenamiento. Es importante destacar que la calidad de las imágenes generadas es excepcional.

Además, los investigadores también adaptaron varias técnicas de acondicionamiento para guiar el proceso de difusión utilizando indicaciones de texto, imágenes o cualquier otra entrada. Esto hace posible producir rápidamente una imagen hermosa y de alta resolución de una salamandra leyendo un libro, o cualquier otra cosa que te guste. También puedes condicionar el proceso de generación de imágenes usando una imagen de entrada. Esto permite muchas aplicaciones, como la pintura exterior, donde una imagen de entrada se extiende más allá de sus bordes, o la pintura exterior, donde se llenan los agujeros en una imagen.

Por último, un potente modelo de difusión latente preentrenado llamado Stable Diffusion fue de código abierto en agosto de 2022 por una colaboración entre LMU Munich y algunas empresas, incluidas StabilityAI y Runway, con el apoyo de EleutherAI y LAION. En septiembre de 2022, fue portado a TensorFlow y incluido en KerasCV, una biblioteca de visión por ordenador creada por el equipo de Keras. Ahora cualquiera puede generar imágenes alucinantes en segundos, de forma gratuita, incluso en un portátil normal (ver el último ejercicio de este capítulo). ¡Las posibilidades son infinitas!

En el próximo capítulo pasaremos a una rama completamente diferente del aprendizaje profundo: el aprendizaje por refuerzo profundo.

# Ejercicios

1. ¿Cuáles son las principales tareas para las que se utilizan los autocodificadores?

2. Supongamos que quieres entrenar a un clasificador y tienes muchos datos de entrenamiento sin etiquetar, pero solo unos pocos miles de instancias etiquetadas. ¿Cómo pueden ayudar los autocodificadores? ¿Cómo procederías?

3. Si un autocodificador reconstruye perfectamente las entradas, ¿es necesariamente un buen autocodificador? ¿Cómo se puede evaluar el rendimiento de un autocodificador?

4. ¿Qué son los autocodificadores incompletos y demasiado completos? ¿Cuál es el principal riesgo de un autocodificador excesivamente insuficiente? ¿Qué pasa con el principal riesgo de un autocodificador demasiado completo?

5. ¿Cómo se atan los pesos en un autocodificador apilado? ¿Qué sentido tiene hacerlo?

6. ¿Qué es un modelo generativo? ¿Puedes nombrar un tipo de autocodificador generativo?

7. ¿Qué es un GAN? ¿Puedes nombrar algunas tareas en las que los GAN pueden brillar?

8. ¿Cuáles son las principales dificultades al entrenar a los GAN?

9. ¿Qué son buenos los modelos de difusión? ¿Cuál es su principal limitación?

10. Intente usar un autocodificador de denodo para preentrenar un clasificador de imágenes. Puedes usar MNIST (la opción más simple) o un conjunto de datos de imágenes más complejo como CIFAR10 si quieres un desafío más grande. Independientemente del conjunto de datos que estés utilizando, sigue estos pasos:

- Divida el conjunto de datos en un conjunto de entrenamiento y un conjunto de pruebas. Entrena a un autoencoder de desnido profundo en el conjunto de entrenamiento completo.

- Comprueba que las imágenes estén bastante bien reconstruidas. Visualiza las imágenes que más activan cada neurona en la capa de codificación.

- Construye una clasificación DNN, reutilizando las capas inferiores del autoencoder. Entrenalo usando solo 500 imágenes del conjunto de entrenamiento. ¿Funciona mejor con o sin entrenamiento previo?

11. Entrena un autocodificador variacional en el conjunto de datos de imágenes de tu elección y úsalo para generar imágenes. Alternativamente, puede intentar encontrar un conjunto de datos sin etiquetar que le interese y ver si puede generar nuevas muestras.

12. Entrena a un DCGAN para abordar el conjunto de datos de imágenes de tu elección y úsalo para generar imágenes. Añade la repetición de la experiencia y mira si esto ayuda. Consitelo en un GAN condicional donde puedes controlar la clase generada.

13. Revisa el excelente tutorial de difusión estable de KerasCV y genera un hermoso dibujo de una salamandra leyendo un libro. Si publicas tu mejor dibujo en Twitter, por favor, etiquétame en @aureliengeron. ¡Me encantaría ver tus creaciones!

Las soluciones a estos ejercicios están disponibles al final del cuaderno de este capítulo, en https://homl.info/colab3.